In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from scipy.stats import trim_mean
from numpy import mean, absolute
import multiprocessing

In [2]:
data = pd.read_csv('sin_nulos.csv')
print(data.size)
data.head()

1739375


,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2009-01-01,Cobar,17.9,35.2,0.0,12.0,12.3,SSW,48.0,ENE,...,20.0,13.0,1006.3,1004.4,2.0,5.0,26.6,33.4,No,No
1,2009-01-02,Cobar,18.4,28.9,0.0,14.8,13.0,S,37.0,SSE,...,30.0,8.0,1012.9,1012.1,1.0,1.0,20.3,27.0,No,No
2,2009-01-03,Cobar,15.5,34.1,0.0,12.6,13.3,SE,30.0,N,...,68.0,7.0,1017.4,1011.6,5.0,1.0,17.1,32.7,No,No
3,2009-01-04,Cobar,19.4,37.6,0.0,10.8,10.6,NNE,46.0,NNE,...,42.0,22.0,1012.3,1009.2,1.0,6.0,28.7,34.9,No,No
4,2009-01-05,Cobar,21.9,38.4,0.0,11.4,12.2,WNW,31.0,WNW,...,37.0,22.0,1012.7,1009.1,1.0,5.0,29.1,35.6,No,No


### ORIGINAL

In [21]:
def similitud_categorica(df):
    matrix_ = df.to_numpy()
    n = matrix_.shape[0]
    m = matrix_.shape[1]
    similitud_categorica =np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            similitud_categorica[i][j]=(matrix_[i]==matrix_[j]).sum()/m
    return similitud_categorica

def similitud_numerica(df):
    n = df.shape[0]
    df_numerica= df.select_dtypes(include='float64')
    range_ = df_numerica.max()-df_numerica.min()
    distancia =np.zeros((n,n))
    D = 0
    for i in range(n):
        for j in range(n):
            diff = (df.loc[i]-df.loc[j])/range_
            distancia[i][j] = np.sqrt((diff**2).sum())
            D += distancia[i][j] 
    D = D/(n*n)
    alpha = -np.log(0.5)/D
    similitud = np.exp(-alpha*distancia)
    return similitud

def entropia_one_type(similitud):
    entropia_total = 0
    for i in range(similitud.shape[0]-1):
        for j in range(i+1,similitud.shape[0]):
            if similitud[i][j]!=0 and similitud[i][j]!=1:
                entropia_total -= (similitud[i][j]*np.log(similitud[i][j]))+((1-similitud[i][j])*np.log(1-similitud[i][j]))
    return entropia_total

def entropia_total(categorica,numerica,c=True,n=True):
    if c:
        s_c = similitud_categorica(categorica)
    
        entropia_c = entropia_one_type(s_c)
    else:
        entropia_c = 0
    if n : 
        s_n = similitud_numerica(numerica)
    
        entropia_n = entropia_one_type(s_n)
    else:
        entropia_n = 0
    entropia_total = entropia_c + entropia_n
    return entropia_total

def ranking_entropia(df):
    
    lista_inicial = list(df.keys())
    ranking = []

    while len(lista_inicial)>1: 
        differences = []
        nueva_df = df.loc[:,lista_inicial]
        numerica= nueva_df.select_dtypes(include='float64')
        categorica = nueva_df.select_dtypes(include='object')
        E_f = entropia_total(categorica,numerica)
        for n in lista_inicial:
            partial_df = nueva_df.loc[:, nueva_df.columns != n]
            numerica= partial_df.select_dtypes(include='float64')
            categorica = partial_df.select_dtypes(include='object')
            if numerica.shape[1]!=0 and categorica.shape[1]!=0:  
                E_f_nueva = entropia_total(categorica,numerica)
                diff = abs(E_f - E_f_nueva)
            elif numerica.shape[1]!=0:
                E_f_nueva = entropia_total(None,numerica,c=False,n=True)
                diff = abs(E_f - E_f_nueva)
            elif categorica.shape[1]!=0:
                E_f_nueva = entropia_total(categorica,None,c=True,n=False)
                diff = abs(E_f - E_f_nueva)
            differences.append(diff)
        # print('Variables : ', lista_inicial)
        # print('Diferencias de entropia : ',differences)
        element = differences.index(min(differences))
        ranking.append(lista_inicial.pop(element))
    return ranking

df_copia = data.copy()
ranking = ranking_entropia(df_copia.head(20).reset_index())
print(f'El ranking de las variables encontrado es, en orden ascendente :{ranking}')

El ranking de las variables encontrado es, en orden ascendente :['index', 'Temp9am', 'Rainfall', 'Humidity3pm', 'Pressure3pm', 'Cloud3pm', 'MinTemp', 'Cloud9am', 'Pressure9am', 'MaxTemp', 'Sunshine', 'WindGustSpeed', 'Humidity9am', 'Date', 'WindSpeed3pm', 'RainToday', 'WindGustDir', 'Temp3pm', 'RainTomorrow', 'WindDir3pm', 'WindSpeed9am', 'Evaporation', 'Location']


C:\Users\juanc\AppData\Local\Temp\ipykernel_15620\4235315234.py:23: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = -np.log(0.5)/D
C:\Users\juanc\AppData\Local\Temp\ipykernel_15620\4235315234.py:24: RuntimeWarning: invalid value encountered in multiply
  similitud = np.exp(-alpha*distancia)


### PARALELIZADO

In [22]:
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


def entropia_one_type(similitud):
    entropia_total = 0
    for i in range(similitud.shape[0]-1):
        for j in range(i+1,similitud.shape[0]):
            if similitud[i][j]!=0 and similitud[i][j]!=1:
                entropia_total -= (similitud[i][j]*np.log(similitud[i][j]))+((1-similitud[i][j])*np.log(1-similitud[i][j]))
    return entropia_total

def similitud_categorica(df):
    matrix_ = df.to_numpy()
    n = matrix_.shape[0]
    m = matrix_.shape[1]
    similitud_categorica =np.zeros((n,n))
    for i in range(n):
        for j in range(n):
            similitud_categorica[i][j]=(matrix_[i]==matrix_[j]).sum()/m
    return similitud_categorica

def similitud_numerica(df):
    n = df.shape[0]
    df_numerica= df.select_dtypes(include='float64')
    range_ = df_numerica.max()-df_numerica.min()
    distancia =np.zeros((n,n))
    D = 0
    for i in range(n):
        for j in range(n):
            diff = (df.loc[i]-df.loc[j])/range_
            distancia[i][j] = np.sqrt((diff**2).sum())
            D += distancia[i][j] 
    D = D/(n*n)
    alpha = -np.log(0.5)/D
    similitud = np.exp(-alpha*distancia)
    return similitud

def entropia_total(categorica,numerica,c=True,n=True):
    if c:
        s_c = similitud_categorica(categorica)
    
        entropia_c = entropia_one_type(s_c)
    else:
        entropia_c = 0
    if n : 
        s_n = similitud_numerica(numerica)
    
        entropia_n = entropia_one_type(s_n)
    else:
        entropia_n = 0
    entropia_total = entropia_c + entropia_n
    return entropia_total

def ranking_entropia(df):
    
    lista_inicial = list(df.keys())
    ranking = []

    while len(lista_inicial) > 1:
        differences = []
        nueva_df = df.loc[:, lista_inicial]
        numerica = nueva_df.select_dtypes(include='float64')
        categorica = nueva_df.select_dtypes(include='object')
        E_f = entropia_total(categorica, numerica)
        
        def calcular_diferencia(n):
            partial_df = nueva_df.loc[:, nueva_df.columns != n]
            numerica = partial_df.select_dtypes(include='float64')
            categorica = partial_df.select_dtypes(include='object')
            if numerica.shape[1] != 0 and categorica.shape[1] != 0:
                E_f_nueva = entropia_total(categorica, numerica)
                diff = abs(E_f - E_f_nueva)
            elif numerica.shape[1] != 0:
                E_f_nueva = entropia_total(None, numerica, c=False, n=True)
                diff = abs(E_f - E_f_nueva)
            elif categorica.shape[1] != 0:
                E_f_nueva = entropia_total(categorica, None, c=True, n=False)
                diff = abs(E_f - E_f_nueva)
            return diff
        
        with ThreadPoolExecutor(max_workers=8) as executor:
            differences = list(executor.map(calcular_diferencia, lista_inicial))

        element = differences.index(min(differences))
        ranking.append(lista_inicial.pop(element))
    return ranking

# Resto del código sin cambios

df_copia = data.copy()
ranking = ranking_entropia(df_copia.head(20).reset_index())
print(f'El ranking de las variables encontrado es, en orden ascendente :{ranking}')


El ranking de las variables encontrado es, en orden ascendente :['index', 'Temp9am', 'Rainfall', 'Humidity3pm', 'Pressure3pm', 'Cloud3pm', 'MinTemp', 'Cloud9am', 'Pressure9am', 'MaxTemp', 'Sunshine', 'WindGustSpeed', 'Humidity9am', 'Date', 'WindSpeed3pm', 'RainToday', 'WindGustDir', 'Temp3pm', 'RainTomorrow', 'WindDir3pm', 'WindSpeed9am', 'Evaporation', 'Location']


C:\Users\juanc\AppData\Local\Temp\ipykernel_15620\3114748329.py:36: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = -np.log(0.5)/D
C:\Users\juanc\AppData\Local\Temp\ipykernel_15620\3114748329.py:37: RuntimeWarning: invalid value encountered in multiply
  similitud = np.exp(-alpha*distancia)
